#                         BM471 GÖRÜNTÜ İŞLEMEYE GİRİŞ
##        Araç Görüntüsünden Plakayı ve Plaka Numarasını Ayrıştırma Projesi


###  Gerekli Kütüphanelrin Import Edilmesi

In [ ]:
import cv2
import matplotlib.pyplot as plt 
import os
import glob
import numpy as np
import imutils
import easyocr
import pandas as pd

### Araç Plaka Veri Setinin Okunması ve Görüntü İşleme

In [ ]:
path = glob.glob("C:/Users/usta_/Desktop/Untitled Folder/images/*.png")
        
img_array = []

for file in path:
    img = cv2.imread(file) 
    gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_array.append(gray) #resmi img_array dizisine eklenir 

In [ ]:
print(len(img_array)) #433 adet resim vardır

In [ ]:
for i in range(0, len(img_array)):
    temp = img_array[i]
    plt.imshow(cv2.cvtColor(temp,cv2.COLOR_BGR2RGB))
    plt.show()
    

### 1- Filtreleme ve Kenarların Bulunması Adımı

In [ ]:
edged_array = [] #edged array

for i in range(0, len(img_array)):
    temp = img_array[i]
    dfilter = cv2.bilateralFilter(temp, 11, 17, 17) #bilateral filtre uygulanıyor
    edged = cv2.Canny(dfilter, 30, 200) #canny algoritmasıyla çizgiler çıkarılıyor
    edged_array.append(edged)

In [ ]:
for i in range(0, len(edged_array)):
    temp = edged_array[i]
    plt.imshow(cv2.cvtColor(temp, cv2.COLOR_BGR2RGB))
   # plt.imshow(temp, cmap="gray")
    plt.show()

### 2- Görüntünün Kontor Yapısının Çıkarılması

In [ ]:
contour_array = []

for i in range(0, len(edged_array)):
    temp = edged_array[i]
    keypoints = cv2.findContours(temp.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    contour_array.append(contours)
    

In [ ]:
loc_array = []

for i in range(0, len(contour_array)):
    location = None
    for con in contour_array[i]:
        approx = cv2.approxPolyDP(con, 10, True)
        if len(approx) == 4:
            location = approx
            break
    loc_array.append(location)

In [ ]:
len(loc_array)

### 3- Maskeleme

In [ ]:
mask_array = []
mask_array_2 = []

for i in range(0, len(img_array)):
    if np.any(loc_array[i] == None) == False:      
        mask = np.zeros(img_array[i].shape, np.uint8)
        new_image = cv2.drawContours(mask, [loc_array[i]], 0, 255, -1)
        new_image = cv2.bitwise_and(img_array[i], img_array[i], mask=mask)
        mask_array.append(new_image)
        mask_array_2.append(mask)

In [ ]:
for i in range(0, len(mask_array)):
    temp = mask_array[i]
    plt.imshow(temp, cmap="gray")
    plt.show()

In [ ]:
print(len(mask_array))

### 4- Görüntüden Kırpılmış Plaka Görüntüsünün Elde Edilmesi

In [ ]:
cropped_array = []                         #coordinates of mask section include plate

for i in range(0, len(mask_array)):
    temp = img_array[i]
    (x, y) = np.where(mask_array_2[i]==255) 
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = temp[x1:x2 + 1, y1:y2 + 1]
    cropped_array.append(cropped_image)

In [ ]:
for i in range(0, len(cropped_array)):
    temp = cropped_array[i]
    plt.imshow(temp, cmap="gray")
    plt.show()

In [ ]:
len(cropped_array)

In [ ]:
import torch
torch.cuda.empty_cache()

### 5- EasyOCR Kullanımı

In [ ]:
text_array = []

for i in range(0,14):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(cropped_array[i])
    text_array.append(result)
    

In [ ]:
new_array=[]

for i in range(0, len(text_array)):
    result=text_array[i]
    
    if len(result)==0:
        text='null'
    else:
        text = result[0][-2]
        
    new_array.append(text)
    print("Detected lisance plate number is: ",text )
    

In [ ]:
text_array

### 6- Plaka Yazılarının Excel Dosyasına Aktarımı

In [ ]:
listOfPlates = pd.DataFrame()
for i in range(0, len(new_array)):
    temp = pd.DataFrame({'Plates':[new_array[i]]})
    listOfPlates=listOfPlates.append(temp)

In [ ]:
writer = pd.ExcelWriter('plates.xlsx')
listOfPlates.to_csv('plates.csv',sep=',',index=False)
writer.save()


######    Talha Yusuf İnan 181180040       
######    Şeyma Usta 1811800073               
######    Ceren Umay Özten 181180060
